In [ ]:
!pip install transformers sentencepiece
!pip install bitsandbytes
!pip install -U bitsandbytes
!pip install trl
!pip install peft

import os
import torch
import pandas as pd
from google.colab import files
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)

from peft import LoraConfig

from trl import SFTTrainer
from transformers import DataCollatorForLanguageModeling
from datasets import Dataset
from huggingface_hub import login

from datasets import load_dataset
login("//yourlogin")

ds = load_dataset("wikimedia/wikipedia", "20231101.tr")

In [ ]:
from google.colab import drive, files
import pandas as pd
import re
import os

# Google Drive'ı bağla
drive.mount('/content/drive')

Mounted at /content/drive


In [45]:
df = ds['train'].to_pandas()
print(df.head())
df = df.drop(df.columns[:2], axis=1)
df = df.tail(1000).reset_index(drop=True)
df
df["title"]=df["title"]+" hakkında bilgi verir misin ?"
df.columns=["question","answer"]
df.to_csv("data.csv",index=False)


   id                                                url                title  \
0  10         https://tr.wikipedia.org/wiki/Cengiz%20Han           Cengiz Han   
1  16  https://tr.wikipedia.org/wiki/Film%20%28anlam%...  Film (anlam ayrımı)   
2  22      https://tr.wikipedia.org/wiki/Mustafa%20Suphi        Mustafa Suphi   
3  24                https://tr.wikipedia.org/wiki/Linux                Linux   
4  30       https://tr.wikipedia.org/wiki/Bol%C5%9Fevizm            Bolşevizm   

                                                text  
0  Cengiz Han (doğum adıyla Temuçin,  – 18 Ağusto...  
1  Film şu anlamlara gelebilir:\n\n Camlara yapış...  
2  Mehmed Mustafa Subhi (), kısaca Mustafa Suphi,...  
3  Linux (telaffuz: Lin-uks); Linux çekirdeğine d...  
4  Bolşevik, çoğunluktan yana anlamına gelen Rusç...  


In [ ]:
def clean_text(text):
    """Metindeki gereksiz boşlukları ve satır sonlarını temizler."""
    if pd.isna(text):
        return ""

    text = str(text)
    cleaned = text.strip()
    cleaned = re.sub(r'\s+', ' ', cleaned)
    cleaned = re.sub(r'\n+', ' ', cleaned)
    cleaned = re.sub(r'\t+', ' ', cleaned)

    return cleaned.strip()

def clean_dataframe(df):
    """DataFrame'deki tüm sütunlardaki metinleri temizler."""
    cleaned_df = df.copy()

    for col in cleaned_df.columns:
        cleaned_df[col] = cleaned_df[col].apply(clean_text)

    return cleaned_df

def convert_to_llama_format(df, question_col, answer_col):
    """DataFrame'i LLAMA formatına dönüştürür."""
    formatted_data = []

    for _, row in df.iterrows():
        formatted_text = f"<s>[INST] {row[question_col]} [/INST] {row[answer_col]} </s>"
        formatted_data.append(formatted_text)

    return formatted_data


In [ ]:
def process_colab_pipeline():
    """
    Colab'da çalışan tam işlem akışı:
    1. Kullanıcıdan CSV yüklemesini ister
    2. Verileri temizler
    3. Temizlenmiş CSV'yi indirir
    4. LLAMA formatına dönüştürür
    5. LLAMA formatını indirir
    """
    print("Lütfen CSV dosyanızı yükleyin...")
    uploaded = files.upload()

    # Yüklenen dosyanın adını al
    input_filename = list(uploaded.keys())[0]

    # CSV'yi oku
    print(f"CSV okunuyor: {input_filename}")
    df = pd.read_csv(input_filename)
    print(df.tail())

    # Sütun isimlerini göster ve kullanıcıdan seçim iste
    print("\nCSV'nizdeki sütunlar:")
    for idx, col in enumerate(df.columns):
        print(f"{idx}: {col}")

    # Kullanıcıdan sütun seçimlerini al
    question_col = df.columns[int(input("Soru sütununun numarasını girin: "))]
    answer_col = df.columns[int(input("Cevap sütununun numarasını girin: "))]

    # Verileri temizle
    print("\nVeriler temizleniyor...")
    cleaned_df = clean_dataframe(df)

    # Temizlenmiş CSV'yi kaydet ve indir
    cleaned_csv = "temiz_" + input_filename
    cleaned_df.to_csv(cleaned_csv, index=False)
    print(f"\nTemizlenmiş CSV indiriliyor: {cleaned_csv}")
    files.download(cleaned_csv)

    # LLAMA formatına dönüştür
    print("\nVeriler LLAMA formatına dönüştürülüyor...")
    llama_formatted = convert_to_llama_format(cleaned_df, question_col, answer_col)

    # LLAMA formatını dosyaya kaydet ve indir
    llama_output = "llama_format.txt"
    with open(llama_output, 'w', encoding='utf-8') as f:
        for example in llama_formatted:
            f.write(example + '\n')

    print(f"\nLLAMA format dosyası indiriliyor: {llama_output}")
    files.download(llama_output)

    print("\nİşlem tamamlandı!")
    print(f"Toplam işlenen örnek sayısı: {len(cleaned_df)}")

    return cleaned_df, llama_formatted

In [ ]:
temiz_df, llama_verileri = process_colab_pipeline()